In [1]:
import numpy as np
import pandas as pd
import pickle
import seaborn as sns
import scipy
import torch
from torch import nn
from torch.utils import data
from torch.nn.utils.weight_norm import weight_norm
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve
import attention_classification_v5
from attention_classification_v5 import padding, CNN, Encoder, CNN_dataset, ATT_dataset


In [2]:
model = Encoder(d_model=768, d_k=256, n_heads=8, d_v=256, out_dim=1, max_len=604, sub_dim=256, d_ff=1024)
model.double()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
#optimizer = torch.optim.SGD(model.parameters(),lr=0.0001)
critation = nn.MSELoss()
scaler = torch.cuda.amp.GradScaler()
model.cuda()

Encoder(
  (layers): EncoderLayer(
    (enc_self_attn): MultiHeadAttentionwithonekey(
      (W_Q): Linear(in_features=768, out_features=2048, bias=False)
      (W_K): Linear(in_features=768, out_features=256, bias=False)
      (W_V): Linear(in_features=768, out_features=2048, bias=False)
      (fc): Linear(in_features=2048, out_features=1, bias=False)
    )
    (pos_ffn): PoswiseFeedForwardNet(
      (fc): Sequential(
        (0): Linear(in_features=860, out_features=1024, bias=False)
        (1): ReLU()
        (2): Linear(in_features=1024, out_features=1, bias=False)
      )
    )
  )
)

In [3]:
embedding = pickle.load( open( "Y04E_all_X_y.p", "rb" ) )
embedding.keys()

dict_keys(['X_all_hidden', 'X_substrate_encoding', 'y_data'])

In [4]:
x_train,x_test,y_train,y_test = train_test_split(list(zip(embedding["X_all_hidden"],embedding["X_substrate_encoding"])), embedding["y_data"], test_size=0.3, random_state=42)
x_validation,x_test,y_validation,y_test = train_test_split(x_test,y_test,test_size=0.3333, random_state=42)

In [5]:
x_emb_train, x_sub_train = zip(*x_train)
x_emb_validation, x_sub_validation = zip(*x_validation)
x_emb_test, x_sub_test = zip(*x_test)


In [6]:
emb_train = ATT_dataset(list(x_emb_train),list(x_sub_train),y_train,604)
train_loader = data.DataLoader(emb_train,32,True,collate_fn=emb_train.collate_fn)
emb_validation = ATT_dataset(list(x_emb_validation),list(x_sub_validation),y_validation,604)
validation_loader = data.DataLoader(emb_validation,128,False,collate_fn=emb_validation.collate_fn)
emb_test = ATT_dataset(list(x_emb_test),list(x_sub_test),y_test,604)
test_loader = data.DataLoader(emb_test,128,False,collate_fn=emb_test.collate_fn)

In [7]:
for epoch in range(100):#1500
    model.train()
    for seq in train_loader:
        enc_inputs, input_mask, substrate, target = seq["embedding"], seq["mask"], seq["substrate"].double(), seq["target"]
        enc_inputs, input_mask, substrate, target = enc_inputs.cuda(), input_mask.cuda(), substrate.cuda(), target.cuda()
        with torch.cuda.amp.autocast():
            output, _ = model(enc_inputs, input_mask, substrate)
            loss = critation(output,target.view(-1,1))
        """
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        """
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
    model.eval()
    pre = []
    gt = []
    for seq in validation_loader:
        enc_inputs, input_mask, substrate, target = seq["embedding"], seq["mask"], seq["substrate"].double(), seq["target"]
        enc_inputs, input_mask, substrate = enc_inputs.cuda(), input_mask.cuda(), substrate.cuda()
        with torch.no_grad():
            output = model(enc_inputs, input_mask, substrate)
        output = output.cpu().detach().numpy().reshape(-1)
        target = target.numpy()
        pre.append(output)
        gt.append(target)
    pre = np.concatenate(pre)
    gt = np.concatenate(gt)
    slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(pre,gt)
    print("epoch: {} | loss: {} | valiloss: {}".format(epoch,loss,r_value))

KeyboardInterrupt: 